В этом буке проведу работу, подобную проделанной с Cys, но с учетом не точностей и не оптимальности предыдущей работы 

### Что есть на данный момент

1) Финальные полученные последовательности для белков 

2) Результаты pfam, полученные для всех, не только отобранных более тщательно

In [6]:
import pandas as pd
'''Доступ к Uniprot'''
from Bio import ExPASy
from Bio import SwissProt
from bs4 import BeautifulSoup
from tqdm import trange

In [58]:
with open('Without_Cys/final_proteins_4_cl.fasta', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line.lstrip('>')
        else:
            fasta.setdefault(name, []).append(line)

In [59]:
columns = []
d = []
with open('Without_Cys/pfam_4_cl.txt', 'r') as f:
    for line in f:
        if line.startswith('<'):
            columns = line.split('> <')
        else:
            d.append(line.split())
columns[0] = columns[0].strip('<')
columns[-1] = columns[-1].strip('>\n')

In [60]:
data = pd.DataFrame(d, columns=columns)

In [61]:
data.head()

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan
0,M1FCN8,32,128,22,129,PF13426.7,PAS_9,Domain,11,103,104,76.6,1.5e-21,1,CL0183
1,M1FCN8,148,251,148,251,PF13426.7,PAS_9,Domain,1,104,104,64.4,9.1e-18,1,CL0183
2,M1FCN8,265,416,263,416,PF00990.21,GGDEF,Domain,3,161,161,125.5,1.5e-36,1,CL0276
3,V4Y023,135,236,134,236,PF13426.7,PAS_9,Domain,2,104,104,42.9,4.7e-11,1,CL0183
4,V4Y023,257,361,255,361,PF13426.7,PAS_9,Domain,3,104,104,76.0,2.3e-21,1,CL0183


In [62]:
data.shape

(4085, 15)

In [63]:
data = data[data['seq id'].isin(fasta.keys())].reset_index(drop = True)
data.shape

(897, 15)

In [64]:
data['kingdom'] = None

In [65]:
def kingdom(s):
    handle = ExPASy.get_sprot_raw(s[0])
    try:
        record = SwissProt.read(handle)
        return record.organism_classification[0] + ' ' + record.organism_classification[1]
    except ValueError:
        return None

In [66]:
for i in trange(897):
    data.iloc[i, 15] = kingdom(data.iloc[i,:])

100%|██████████| 897/897 [09:48<00:00,  1.38it/s]  


In [67]:
tfile = open('Without_Cys/domains_4_cl.csv', 'a')
tfile.write(data.to_csv())
tfile.close()